## 아이리스 다중 분류

In [11]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [12]:
import pandas as pd
df = pd.read_csv('dataset/iris.csv', 
                 names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [20]:
dataset = df.values
x_data = dataset[:,0:4]
Y_obj = dataset[:,4]
Y_obj[0], Y_obj[50], Y_obj[100]

('Iris-setosa', 'Iris-versicolor', 'Iris-virginica')

In [30]:
from pandas import get_dummies
y_data = get_dummies(Y_obj)

In [29]:
y_data

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [31]:
print(x_data.shape, y_data.shape)

(150, 4) (150, 3)


In [16]:
# seed 값 설정
seed = 0
tf.set_random_seed(seed)

In [17]:
# 입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float64, shape=[None, 4])
Y = tf.placeholder(tf.float64, shape=[None, 3])

In [22]:
# Layer 1
W1 = tf.Variable(tf.random_normal([4,16], dtype=tf.float64)) 
b1 = tf.Variable(tf.random_normal([16], dtype=tf.float64))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

In [37]:
# Layer 2
W2 = tf.Variable(tf.random_normal([16,1], dtype=tf.float64)) 
b2 = tf.Variable(tf.random_normal([1], dtype=tf.float64))
logits = tf.matmul(L1, W2) + b2

In [39]:
# cross-entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_data))
#cost = -tf.reduce_sum(Y * tf.log(hypo))

In [40]:
learning_rate = 0.1
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [41]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(51):
    _, cost_val = sess.run([train, cost], feed_dict = {X: x_data, Y: y_data})
    if epoch % 10 == 0:
        print("step=%d, \tcost_val=%.5f" % (epoch + 1, cost_val))

InvalidArgumentError: Input to reshape is a tensor with 450 values, but the requested shape has 150
	 [[node gradients_1/softmax_cross_entropy_with_logits_sg/Reshape_grad/Reshape (defined at C:\Users\ckkim\.conda\envs\tf2\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]

Original stack trace for 'gradients_1/softmax_cross_entropy_with_logits_sg/Reshape_grad/Reshape':
  File "C:\Users\ckkim\.conda\envs\tf2\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ckkim\.conda\envs\tf2\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ckkim\.conda\envs\tf2\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ckkim\.conda\envs\tf2\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\ckkim\.conda\envs\tf2\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
 

In [34]:
predict = tf.equal(tf.argmax(y_data, 1), tf.argmax(hypo, 1))
accuracy = tf.reduce_mean(tf.cast(predict, dtype=tf.float64))

In [35]:
acc = sess.run(accuracy, feed_dict = {X: x_data, Y: y_data})
print("Accuracy: %.4f" % acc)

Accuracy: 0.3333


In [13]:
# 딥러닝 모델 설정
model = Sequential([
    Dense(16, input_shape=(4,), activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 모델 컴파일 
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
# 모델 실행
model.fit(X, Y_encoded, epochs=50, batch_size=10)

Train on 150 samples
Epoch 1/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2945 - accuracy: 0.9667
Epoch 2/50
150/150 [==============================] - 0s 220us/sample - loss: 0.2896 - accuracy: 0.9800
Epoch 3/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2861 - accuracy: 0.9733
Epoch 4/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2816 - accuracy: 0.9733
Epoch 5/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2799 - accuracy: 0.9800
Epoch 6/50
150/150 [==============================] - 0s 239us/sample - loss: 0.2748 - accuracy: 0.9667
Epoch 7/50
150/150 [==============================] - 0s 247us/sample - loss: 0.2726 - accuracy: 0.9800
Epoch 8/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2655 - accuracy: 0.9800
Epoch 9/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2635 - accuracy: 0.9733
Epoch 10/50
150/150 [======================

In [19]:
# 결과 출력 
print("\n Accuracy: %.4f" % (model.evaluate(X, Y_encoded, verbose=0)[1]))


 Accuracy: 0.9733
